In [1]:
!pip install bitsandbytes accelerate
!pip install torch transformers
!pip install -U datasets
!pip install -U sentence-transformers
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 117.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 MB 139.9 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 195.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 67.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 105.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.5/718.5 kB 124.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 208.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 39.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 258.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21/21 [accelerate]1 [bitsandbytes]ub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 188.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 273.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/

In [1]:
import pandas as pd
df = pd.read_csv('final.csv')

In [2]:
laliste=[]
laliste_labels=[]
for elt in df['label']:
    laliste_labels.append(elt)
for elt in df['text']:
    laliste.append(elt)

In [3]:
from sentence_transformers import SentenceTransformer
import torch

# Define arguments for 4-bit quantization
model_kwargs = {
    "device_map": "auto",         # Automatically map layers to devices (GPU/CPU)
    "torch_dtype": torch.bfloat16,  # Recommended dtype for Qwen models
    "load_in_4bit": True,           # Enable 4-bit quantization
}

# The model card also recommends enabling flash_attention_2 for better performance
# You may need to install it: pip install flash-attn
# Set use_flash_attention_2=True if available
embedding_model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-8B",
    model_kwargs=model_kwargs,
    trust_remote_code=True, # Often required for custom model architectures
    # use_flash_attention_2=True, # Uncomment if flash-attn is installed
)

/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The 8-bit optimizer is not available on your device, only available on CUDA for now.
Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.23s/it]


In [5]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

In [6]:
documents = laliste[:10]

len(documents)

10

In [7]:
synth_data = torch.from_numpy(embedding_model.encode(documents))

In [8]:

documents = laliste

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
synth_data = torch.from_numpy(embedding_model.encode(documents))
validation_data = torch.from_numpy(embedding_model.encode(validation['cleaned_cite_text'].tolist()))
test_data = torch.from_numpy(embedding_model.encode(test['cleaned_cite_text'].tolist()))

In [9]:
import torch

import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=128, num_classes=6):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)
mymodel = MLPClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=0.001)

In [11]:
from torch.utils.data import TensorDataset, DataLoader
X_train = synth_data
y_train = torch.tensor(laliste_labels)

batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()           # ✅ Reset gradients from previous step
        outputs = mymodel(X_batch)        # Forward pass
        loss = criterion(outputs, y_batch)
        loss.backward()                 # Compute gradients
        optimizer.step()                # Update weights


In [13]:
from sklearn.metrics import classification_report

In [14]:
with torch.no_grad():
    validation_outputs = mymodel(validation_data)
    test_outputs = mymodel(test_data)

In [19]:
torch.argmax(validation_outputs,axis=1)

tensor([3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3,
        3, 2, 3, 3, 2, 3, 3, 3, 5, 3, 5, 3, 3, 1, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3,
        3, 3, 1, 3, 3, 3, 3, 3, 3, 5, 2, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 1, 3, 2, 3, 5, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3])

In [20]:
print(classification_report(validation['intent'], torch.argmax(validation_outputs, axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.00      0.00      0.00        21
           2       0.11      0.05      0.07        21
           3       0.03      0.75      0.06         4
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         4

    accuracy                           0.04       114
   macro avg       0.02      0.13      0.02       114
weighted avg       0.02      0.04      0.01       114



/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is